In [2]:
## Import the libraries
import requests as rq
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options

from selenium.webdriver.common.by import By # to find tags or elements
from selenium.webdriver.support.ui import WebDriverWait # a good way to wait
from selenium.webdriver.support import expected_conditions as EC # for scrolling

import time
import pandas as pd

In [4]:
## Testing url - Jumia
url = 'https://www.jumia.com.ng/catalog/?q=iphone+16&price=118674-3500000&page=1#catalog-listing'
response = rq.get(url)
response.status_code

200

In [5]:
soup = BeautifulSoup(response.text, 'html.parser')

## Scrap Data for Mouse on Jumia

#### To Do
- Search for laptop mouse on Jumia and manually check for results that have good amount of reviews.
    - for this test, I will search for just 1 result with a good number of reviews
- get the url - [mouse link](https://www.jumia.com.ng/catalog/productratingsreviews/sku/GE779EA0A9NPFNAFAMZ/?page=1)
- use the url to scrap all the reviews

In [6]:
## Demo Extraction
base_url = 'https://www.jumia.com.ng'

product_names = soup.find('h3', class_ = 'name').text
product_price = soup.find('div', class_ = 'prc').text
original_price = soup.find('div', class_ = 'old').text
discount = soup.find('div', class_ = 'bdg _dsct _sm').text
product_star = soup.find('div', class_ = 'stars _s').text
product_url = base_url+soup.find('a', class_ = 'core')['href']

In [7]:
product_url

'https://www.jumia.com.ng/apple-iphone-16-8gb-128gb-5g-ultramarine-379768924.html'

In [9]:
pp_url = 'https://www.jumia.com.ng/apple-iphone-16-pro-max-8gb-512gb-5g-black-titanium-379789023.html'

In [10]:
response_2 = rq.get(pp_url)
soup_2 = BeautifulSoup(response_2.text, 'html.parser')

In [11]:
reviewer_header_comment = soup_2.find('h3', class_ = '-m -fs16 -pvs').text
reviewer_detail_comment = soup_2.find('p', class_ = '-pvs').text
review_date = soup_2.find('span', class_ = '-prs').text
div = soup_2.find("div", class_="-df -j-bet -i-ctr -gy5")
reviewer_name = div.find_all('span')[1].text

In [12]:
reviewer_name

'by Truth Oamen'

In [ ]:
## Configure Selenium EdgeDriver options
options = Options()
options.use_chromium = True
service = Service(executable_path=r'Driver\msedgedriver.exe') # Always check to confirm that the version of edge driver matches the version or MS Edge browser

## Initialize the webdriver
driver = webdriver.Edge(service=service, options=options)

## Testing the fetch
product_stars = []
reviewer_header_comments = []
reviewer_detail_comments = []
review_dates = []
reviewer_names = []

page_count = 4

for page in range(1, page_count+1):
    url = f'https://www.jumia.com.ng/catalog/productratingsreviews/sku/GE779EA0A9NPFNAFAMZ/?page={page}'
    driver.get(url)
    time.sleep(20)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    the_page = soup.find_all('article', class_ = '-pvs -hr _bet')

    # Check each phone result on each page
    for item in the_page:
        # Reviewer Header Comment
        try:
            reviewer_header_comment = item.find('h3', class_ = '-m -fs16 -pvs').text
        except AttributeError:
            reviewer_header_comment = ''
        reviewer_header_comments.append(reviewer_header_comment)

        # Reviewer Detail Comment
        try:
            reviewer_detail_comment = item.find('p', class_ = '-pvs').text
        except AttributeError:
            reviewer_detail_comment = ''
        reviewer_detail_comments.append(reviewer_detail_comment)

        # Review Dates
        try:
            review_date = item.find('span', class_ = '-prs').text
        except AttributeError:
            review_date = ''
        review_dates.append(review_date)

        # Reviewer Name
        try:
            div = item.find("div", class_="-df -j-bet -i-ctr -gy5")
            reviewer_name = div.find_all('span')[1].text
        except AttributeError:
            reviewer_name = ''
        reviewer_names.append(reviewer_name)
        
        # product star
        try:
            product_star = item.find('div', class_ = 'stars _m _al -mvs').text
        except AttributeError:
            product_star = ''
        product_stars.append(product_star)       


# Close the driver
driver.quit()

# Create the dataframe
item_data = {
    'reviewer_header_comments' : reviewer_header_comments,
    'reviewer_detail_comments' : reviewer_detail_comments,
    'review_dates' : review_dates,
    'reviewer_names' : reviewer_names,
    'product_stars' : product_stars   
}

item_df = pd.DataFrame(item_data)

In [4]:
item_df.head()

,reviewer_header_comments,reviewer_detail_comments,review_dates,reviewer_names,product_stars
0,Best best,I gave 5 stars because item was superb,10-02-2025,by Hyginus,5 out of 5
1,Very Okay,"Exactly as seen very useful and portable,I don...",10-02-2025,by QUEEN,5 out of 5
2,nice,It's a nice product. I like it,10-02-2025,by Mathew,4 out of 5
3,just as described,just as described,10-02-2025,by Onyeka,5 out of 5
4,Great product,Super enjoyed this... Everything about it is j...,05-02-2025,by Isaac,5 out of 5


In [115]:
airbud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   reviewer_header_comments  31 non-null     object
 1   reviewer_detail_comments  31 non-null     object
 2   review_dates              31 non-null     object
 3   reviewer_names            31 non-null     object
 4   product_stars             31 non-null     object
dtypes: object(5)
memory usage: 1.3+ KB


In [5]:
item_df.shape

(40, 5)

In [119]:
## Saving to csv
airbud_df.to_csv('airbud.csv', index=False)

##### Aliexpress - Air Cooler Fan

In [ ]:
## Configure Selenium EdgeDriver options
options = Options()
options.use_chromium = True
service = Service(executable_path=r'Driver\msedgedriver.exe') # Always check to confirm that the version of edge driver matches the version or MS Edge browser

## Initialize the webdriver
driver = webdriver.Edge(service=service, options=options)

## Testing the fetch
product_stars = []
reviewer_header_comments = []
reviewer_detail_comments = []
review_dates = []
reviewer_names = []

start_url = 'https://www.jumia.com.ng/catalog/productratingsreviews/sku/GE779EA0A9NPFNAFAMZ/?page=1'
driver.get(start_url)
time.sleep(20) # To load the product page

# Clicking the 'View more' button
try:
    view_more_button = driver.find_element(By.CSS_SELECTOR, 'button.v3--btn--KaygomA') # The view more button is a bit tricky
    view_more_button.click() # Click the View more button
    time.sleep(10)  # Wait for the pop-up to load
except Exception as e:
    print("Could not find the 'View more' button:", e)
    driver.quit()
    exit()

# Switch to the review pop-up window
try:
    review_popup = driver.find_element(By.CSS_SELECTOR, 'div[button.v3--btn--KaygomA"]') # The loa
except Exception as e:
    print("Could not find the review pop-up:", e)
    driver.quit()
    exit()


    # review_data.append({
    #     'reviewer_name': reviewer_name,
    #     'review_text': review_text,
    #     'review_date': review_date,
    #     'rating': rating
    # })

# Close the driver
driver.quit()

# reviews_df = pd.DataFrame(review_data)

# # Save to CSV
# reviews_df.to_csv('aliexpress_reviews.csv', index=False)



In [15]:
## Function to get the reviews on each page
def get_page_review():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = soup.find_all('article', class_='-pvs -hr _bet')

    # Extract review data
    review_data = []
    for review in reviews:
        
        # Reviewer Header Comment
        try:
            reviewer_header_comment = review.find('h3', class_ = '-m -fs16 -pvs').text
        except AttributeError:
            reviewer_header_comment = ''
        
        # Reviewer Detail Comment
        try:
            reviewer_detail_comment = review.find('p', class_ = '-pvs').text
        except AttributeError:
            reviewer_detail_comment = ''
        
        # Review Dates
        try:
            review_date = review.find('span', class_ = '-prs').text
        except AttributeError:
            review_date = ''
        
        # Reviewer Name
        try:
            div = review.find("div", class_="-df -j-bet -i-ctr -gy5")
            reviewer_name = div.find_all('span')[1].text
        except AttributeError:
            reviewer_name = ''
                
        # product star
        try:
            product_star = review.find('div', class_ = 'stars _m _al -mvs').text
        except AttributeError:
            product_star = ''
        
        review_data.append({
            'reviewer_name': reviewer_name,
            'reviewer_header_comment': reviewer_header_comment,
            'reviewer_detail_comment': reviewer_detail_comment,
            'review_date': review_date,
            'product_star': product_star
        })
    
    return review_data

In [ ]:
## Configure Selenium EdgeDriver options
options = Options()
options.use_chromium = True
service = Service(executable_path=r'Driver\msedgedriver.exe') # Always check to confirm that the version of edge driver matches the version or MS Edge browser
driver = webdriver.Edge(service=service, options=options) # Initialize the webdriver

# Update the start URL
start_url = 'https://www.jumia.com.ng/catalog/productratingsreviews/sku/GE779EA1BY9XTNAFAMZ/'
driver.get(start_url)
time.sleep(10) # To load the product page

# Scrape all reviews - using a while loop
all_reviews = []

while True:
    d_reviews = get_page_review()
    all_reviews.extend(d_reviews)

    # Clicking the 'View more' button to get to other pages
    try:
        # next_page_button = driver.find_element(By.CSS_SELECTOR, 'a.pg[aria-label="Next Page"]') # The next page button is a bit tricky
        next_page_button = driver.find_element(By.XPATH, '//a[@aria-label="Next Page"]')

        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_page_button) # Scroll the page to view the button for Next Page
        time.sleep(2)  # Wait for the scroll

        next_page_button.click() # Clicks the next page button
        time.sleep(10)  # Wait for the next page to load
    except Exception as e:
        print("No more pages to load")
        break

driver.quit()


# Save to CSV
mouse_df = pd.DataFrame(all_reviews)
mouse_df.to_csv('Jumia__mouse_reviews.csv', index=False)

### Shopify

In [42]:
## Function to get the reviews on each page
def get_page_review():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    first_leg = soup.find('div', class_ = 'flex w-full flex-col gap-space-16 overflow-x-hidden pb-space-32 md:gap-space-24')
    reviews = first_leg.find_all('div', class_='flex flex-col gap-space-16 md:gap-space-24')

    # Extract review data
    review_data = []
    for review in reviews:
        
        # Reviewer Header Comment
        try:
            reviewer_header_comment = review.find('p', class_ = 'font-bodySmallBold text-bodySmallBold').text
        except AttributeError:
            reviewer_header_comment = ''
        
        # Reviewer Detail Comment
        try:
            reviewer_detail_comment = review.find('p', class_ = 'font-bodySmall text-bodySmall text-text').text
        except AttributeError:
            reviewer_detail_comment = ''
        
        # Review Dates - Got index error from the split somewhere between the date and the name. So added a modification
        try:
            date_text = review.find('p', class_ = 'font-caption text-caption text-text-tertiary').text
            date_part = date_text.split('·')
            review_date = date_part[1] if len(date_part) > 1 else ''
        except (AttributeError, IndexError):
            review_date = ''
        
        # Reviewer Name
        try:
            name_text = review.find('p', class_ = 'font-caption text-caption text-text-tertiary').text
            name_part = name_text.split('·')
            reviewer_name = name_part[0] if len(name_part) > 1 else ''
        except (AttributeError, IndexError):
            reviewer_name = ''
                
        # product star
        try:
            rating = review.find('div', class_ = 'flex flex-row items-center gap-space-2 border-border-tertiary')
            product_star = rating['aria-label']
        except AttributeError:
            product_star = ''
        
        review_data.append({
            'reviewer_name': reviewer_name,
            'reviewer_header_comment': reviewer_header_comment,
            'reviewer_detail_comment': reviewer_detail_comment,
            'review_date': review_date,
            'product_star': product_star
        })
    
    return review_data

In [43]:
## Configure Selenium EdgeDriver options
options = Options()
options.use_chromium = True
service = Service(executable_path=r'Driver\msedgedriver.exe') # Always check to confirm that the version of edge driver matches the version or MS Edge browser
driver = webdriver.Edge(service=service, options=options) # Initialize the webdriver

# Update the start URL
start_url = 'https://shop.app/products/7533915177117'
driver.get(start_url)
time.sleep(15) # To load the product page

# Scrape all reviews - using a while loop
all_reviews = []

# Clicking 'Read more reviews' button
try:
    read_more_reviews_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
        (By.XPATH, '//button[@class="bg-bg-fill-secondary stroke-text text-text hover:bg-bg-fill-secondary-hover focus-visible:outline-none focus-visible:ring focus-visible:ring-border-input-active focus-visible:ring-offset-2 false text-buttonMedium font-buttonMedium rounded-radius-10 _button_m_95bul_61 min-w-[72px] p-space-8 transition active:scale-[0.99] w-full relative"]'))
    )
    
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", read_more_reviews_button) # Scroll the page to view the button for Next Page
    time.sleep(2)  # Wait for the scroll
    
    read_more_reviews_button.click() # Click the read more reviews button
    time.sleep(5)
    
    # Then we're changing the driver to now point to the new pop-up window
    reviews_window = WebDriverWait(driver, 10).until(EC.presence_of_element_located(
        (By.XPATH, '//div[@class="flex-1 overflow-y-auto px-screen-margin md:pt-0"]')
    ))

    # reviews_container = (By.XPATH, '//div[@class="flex w-full flex-col gap-space-16 overflow-x-hidden pb-space-32 md:gap-space-24"]')
    # Logic for the scrolling down on the reviews window
    # Get the current scroll height
    # driver.execute_script('arguments[0].scrollBy(0, 2000);', reviews_window)
    scroll_height = driver.execute_script('return arguments[0].scrollHeight', reviews_window)
    
    while True:
        #scroll down little by little by 100 pixels
        driver.execute_script('arguments[0].scrollBy(0, 5000);', reviews_window)
        time.sleep(6)

        new_scroll_height = driver.execute_script('return arguments[0].scrollHeight', reviews_window)
        
        if scroll_height == new_scroll_height:
            break
        scroll_height = new_scroll_height
    
    d_reviews = get_page_review()
    all_reviews.extend(d_reviews)

except Exception as e:
    print("No more pages to load", e)


driver.quit()


# Save to CSV
keyboard_df = pd.DataFrame(all_reviews)
keyboard_df.to_csv('Shopify_keyboard_reviews.csv', index=False)

### Ebay - Laptop Stand

In [28]:
def get_ebay_reviews():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    feedback_container = soup.find('ul', class_ = "fdbk-detail-list__cards")
    all_feedbacks = feedback_container.find_all('li', class_ = "fdbk-container")

    ebay_reviews = []

    for feedbacks in all_feedbacks:
        # Review Date
        try:
            review_date = feedbacks.find('span', class_ = "fdbk-container__details__info__divide__time").text
        except AttributeError:
            review_date = ''

        # Reviewer Name
        try:
            name_text = feedbacks.find('div', class_ = "fdbk-container__details__info__username").text
            reviewer_name = name_text.split('-')[0]
        except AttributeError:
            reviewer_name = ''

        # Reviewer Comment
        try:
            reviewer_comment = feedbacks.find('div', class_ = 'fdbk-container__details__comment').text
        except AttributeError:
            reviewer_comment = ''

        ebay_reviews.append({
            'review_date' : review_date,
            'reviewer_name' : reviewer_name,
            'reviewer_comment' : reviewer_comment
        })

    return ebay_reviews

In [32]:
## Configure Selenium EdgeDriver options
options = Options()
options.use_chromium = True
service = Service(executable_path=r'Driver\msedgedriver.exe') # Always check to confirm that the version of edge driver matches the version or MS Edge browser
driver = webdriver.Edge(service=service, options=options) # Initialize the webdriver

# Update the start URL
start_url = 'https://www.ebay.com/itm/166555275619?_skw=laptop+stand&itmmeta=01JKZQE3TFDX10T68HFNRA9FMH&hash=item26c777d163:g:8uIAAOSwtC1lqFW4&itmprp=enc%3AAQAKAAAA8FkggFvd1GGDu0w3yXCmi1cNeEe876Fw1r5%2BC2QsoNGw6ErEP43kWGJNgUJ1whfWByXJ34tYfFiyG%2BrH9DGnWDNccYEJr5gjtuQdJ2v2qhPUvzIiFJbzkPq0Cvk7CukZ5gNjPTguVw8nDIPM7AyWd7K3l78L0L2dYSUdOs3fUubyb2XBXjcszVTQcApU5VpQ5XXYzgqcus3qnNWwRvt9f6b16x2RQw4iS%2FCbOTLKzuCpYeYbVxFTI%2BB3F2jGzUBAkAyEQ4%2Bdm4is%2BOhQ1kB9HHfKOEngoPmgWe8%2FAFUnNXyXs9AMCSclav4YifXRARFg8w%3D%3D%7Ctkp%3ABFBMwr24959l'
driver.get(start_url)
time.sleep(8) # To load the product page

# Get the 'see more reviews' button
all_feedback_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
    (By.XPATH, '//a[@class="fdbk-detail-list__tabbed-btn fake-btn fake-btn--large fake-btn--secondary"]')
))

# scroll down to where the button is
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", all_feedback_button) # Scroll the page to view the button for Next Page
time.sleep(2)  # Wait for the scroll

# click the button
all_feedback_button.click()
time.sleep(5)

# Change driver focus to point to new page
driver.switch_to.window(driver.window_handles[1])
time.sleep(3)

all_product_reviews = []

while True:
    d_reviews = get_ebay_reviews()
    all_product_reviews.extend(d_reviews)

    try:
        # Get the next page button
        next_page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
            (By.XPATH, '//a[@class="pagination__next icon-link" and @aria-label="Next page"]')
        ))
        # driver.find_element(By.XPATH, '//a[@aria-label="Next Page"]')

        # Scroll to the next page button area
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_page_button)
        time.sleep(2)  # Wait for the scroll

        next_page_button.click() # Clicks the next page button
        time.sleep(5)  # Wait for the next page to load

    except Exception as e:
        print("No more pages to load")
        break

driver.quit()

# Save to csv
ebay_df = pd.DataFrame(all_product_reviews)
ebay_df.to_csv('ebay_laptop_stand_reviews.csv', index=False)

No more pages to load


In [25]:
ebay_df.head(12)

,review_date,reviewer_name,reviewer_comment
0,Past 6 months,l***c (583)- Feedback left by buyer.,"Perfect for what i needed!! Built strong, grea..."
1,Past 6 months,s***s (142)- Feedback left by buyer.,"It works great, however, the adjusting bolts a..."
2,Past 6 months,8***d (50)- Feedback left by buyer.,"These laptop stands are awesome, very sturdy. ..."
3,Past 6 months,2***2 (904)- Feedback left by buyer.,"The stand is stiff, as it should be. I've had ..."
4,Past 6 months,9***n (219)- Feedback left by buyer.,Very strong and sturdy. It's great. Works perf...
5,Past 6 months,d***l (105)- Feedback left by buyer.,Product arrived quickly. The tool to loosen/ti...
6,Past year,u***r (75)- Feedback left by buyer.,Great item high enough for my laptop. Strong m...
7,Past 6 months,5***t (501)- Feedback left by buyer.,Sturdy product. Holds up really well. Satisfie...
8,Past 6 months,r***i (666)- Feedback left by buyer.,Envío rápido y eficiente. Artículo en excelent...
9,Past 6 months,p***1 (16)- Feedback left by buyer.,"It’s a little hard to position, but it stays i..."


In [26]:
ebay_df.shape

(25, 3)

In [15]:
# Update the start URL
start_url = 'https://www.ebay.com/itm/166555275619?_skw=laptop+stand&itmmeta=01JKZQE3TFDX10T68HFNRA9FMH&hash=item26c777d163:g:8uIAAOSwtC1lqFW4&itmprp=enc%3AAQAKAAAA8FkggFvd1GGDu0w3yXCmi1cNeEe876Fw1r5%2BC2QsoNGw6ErEP43kWGJNgUJ1whfWByXJ34tYfFiyG%2BrH9DGnWDNccYEJr5gjtuQdJ2v2qhPUvzIiFJbzkPq0Cvk7CukZ5gNjPTguVw8nDIPM7AyWd7K3l78L0L2dYSUdOs3fUubyb2XBXjcszVTQcApU5VpQ5XXYzgqcus3qnNWwRvt9f6b16x2RQw4iS%2FCbOTLKzuCpYeYbVxFTI%2BB3F2jGzUBAkAyEQ4%2Bdm4is%2BOhQ1kB9HHfKOEngoPmgWe8%2FAFUnNXyXs9AMCSclav4YifXRARFg8w%3D%3D%7Ctkp%3ABFBMwr24959l'
response = rq.get(start_url)
# time.sleep(8) # To load the product page

# Get the 'see more reviews' button
# all_feedback_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
#     (By.XPATH, '//a[@class="fdbk-detail-list__tabbed-btn fake-btn fake-btn--large fake-btn--secondary"]')
# ))

soup = BeautifulSoup(response.text, 'html.parser')
another_url = soup.find('a', class_ = "fdbk-detail-list__tabbed-btn fake-btn fake-btn--large fake-btn--secondary")


In [19]:
another_url['href']

'https://www.ebay.com/fdbk/mweb_profile?fdbkType=FeedbackReceivedAsSeller&item_id=166555275619&username=mestar_decor&filter=feedback_page%3ARECEIVED_AS_SELLER&q=166555275619&sort=RELEVANCE'